In [ ]:
import os
import openai
import anthropic
import together
import subprocess
import re

# open ai API key
openai_api_key='    '

# anthropic API key
claude_api_key = ''

# together AI API key
together_api_key = ''

In [7]:
# read the prompt from the prompt text file titled "dynamnews.txt"
with open("../prompts/dynamnews.txt", "r") as f:
    prompt = f.read()

prompt = str(prompt)

In [ ]:
print(prompt)

In [3]:
claude_client = anthropic.Anthropic(api_key = claude_api_key)
openai_client = openai.OpenAI(api_key = openai_api_key)
togetherai_client = together.Together(api_key=together_api_key)

## GPT 4o

In [9]:
completion = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are an expert in probability theory and stochastic modeling."},
        {"role": "user", "content": prompt},
    ]
)

# get the response
response = completion.choices[0].message.content

In [10]:
print(response)

To tackle the problem of determining the optimal initial inventory levels \( x = (x_1, ..., x_n) \) for maximizing expected profit, we can approach this problem using a simulation-based method. Given the complexity of the Multinomial Logit (MNL) model and the discrete nature of customer choices based on random utilities, deriving a closed-form solution analytically is challenging. Therefore, a simulation approach with a structured optimization search over initial inventory levels is more feasible here.

### Simulation-Based Approach with Optimization

**Objective:** Maximize the expected profit defined as the revenue from sold units minus the cost of the initial inventory. 

#### Steps:

1. **Model Setup:**
   - Assume \( n = 10 \) products.
   - Set customer arrival \( T = 30 \).
   - Set base utilities \( u^j = 5 + j \).
   - Gumbel distribution parameters for noise are set with \( \mu = 1.0 \).
   - Prices and costs for products are \( p^j = 9 \) and \( c^j = 5 \).

2. **Simulation 

In [12]:
import numpy as np

# Constants and initial settings
num_prods = 10
num_customers = 30
price = np.array([9] * num_prods)
cost = np.array([5] * num_prods)
base_utilities = np.array([5 + j for j in range(num_prods)])
budget = 1000  # Simulation or replications budget

# Simulating customer choice
def simulate_profit(inventory, seed=0):
    np.random.seed(seed)
    profit = 0
    tracking_inventory = inventory.copy()
    for customer in range(num_customers):
        epsilon = np.random.gumbel(0, 1, num_prods)
        utilities = base_utilities + epsilon
        available = np.where(tracking_inventory > 0, 1, 0)
        available_utilities = available * utilities

        if np.any(available_utilities):
            choice = np.argmax(available_utilities)
            tracking_inventory[choice] -= 1
            profit += price[choice] - cost[choice]

    total_inventory_cost = np.sum(inventory * cost)
    return profit - total_inventory_cost

# Optimization
best_inventory = None
best_profit = -np.inf

# Simple grid search over possible inventory levels
for _ in range(budget):
    inventory = np.random.randint(0, 10, num_prods)
    total_profit = np.mean([simulate_profit(inventory, seed) for seed in range(30)])
    
    if total_profit > best_profit:
        best_profit = total_profit
        best_inventory = inventory

print(f"Optimal Inventory: {best_inventory}")
print(f"Expected Profit: {best_profit}")

Optimal Inventory: [4 7 1 5 9 9 9 7 4 1]
Expected Profit: -160.0


## o1

In [13]:
completion = openai_client.chat.completions.create(
    model="o1",
    messages=[
        {"role": "system", "content": "You are an expert in probability theory and stochastic modeling."},
        {"role": "user", "content": prompt},
    ]
)

# get the response
response = completion.choices[0].message.content

In [14]:
print(response)

Below is one possible solution approach, combining an analytic argument for the special “all‐prices/all‐costs‐equal” case and a simulation‐based method that can be generalized. The key insight is that, because all products have the same margin (price − cost = 4) and product 10 has the highest baseline utility, one can show analytically that stocking only product 10 up to (approximately) the total number of customers T is optimal. We then illustrate how one could confirm this (or search for other solutions) via simulation.

────────────────────────────────────────────────────────────────────────────
1. ANALYTICAL / CLOSED-FORM ARGUMENT
────────────────────────────────────────────────────────────────────────────

■ Setting
• Number of products n = 10.  
• Number of customers T = 30.  
• Base utilities u^j = 5 + j for j = 1,…,10, so product 10 has the highest base utility, u^10 = 15.  
• Price p^j = 9 and cost c^j = 5 for all j. Hence the profit margin for any sold unit is p^j − c^j = 4. 

In [15]:
import math
import random
import statistics

def gumbel_rnd(mu=0.0, beta=1.0):
    """
    Sample from a Gumbel(mu, beta). 
    If beta=1, the cdf is exp(-exp(-(x - mu))).
    We'll just use random.random() and transform.
    """
    u = random.random()
    return mu - beta * math.log(-math.log(u))

def simulate_mnl_choice(x, u, T, price, cost, mu=1.0, seed=None):
    """
    Simulate the profit for a single run (one realization) of T customers.
    
    Inputs:
        x     = list of initial inventory levels [x1, x2, ..., xn].
        u     = list of base utilities [u1, u2, ..., un].
        T     = number of customers.
        price = list of product prices [p1, p2, ..., pn].
        cost  = list of product costs [c1, c2, ..., cn].
        mu    = scale parameter for Gumbel noise (default=1.0).
        seed  = optional random seed for reproducibility.
    
    Returns:
        profit = total profit from this single simulation run.
    """
    if seed is not None:
        random.seed(seed)
    
    n = len(u)
    inventory = x[:]  # copy
    profit = 0.0
    
    # Pre-calculate total cost for holding the chosen inventory x
    total_inventory_cost = sum(cost[j]*inventory[j] for j in range(n))
    
    total_revenue = 0.0
    
    for t in range(T):
        # Identify which products are in stock
        in_stock_indices = [j for j in range(n) if inventory[j] > 0]

        if len(in_stock_indices) == 0:
            # User leaves or no purchase
            continue
        
        # Sample Gumbel noise for each product j
        epsilons = [gumbel_rnd(0.0, mu) for j in range(n)]
        
        # Utilities = u[j] + epsilons[j], but only consider j in stock
        # Also consider no-purchase utility = 0 + gumbel noise(=0 if we prefer exact MNL).
        # For standard MNL with "outside good" at utility=0, we compare
        # Gumbel(0) plus base 0 to products. But we'll do the logit formula directly.
        
        # Argmax approach: pick the product that gives highest (u[j] + epsilons[j]) among in-stock plus 0 for outside.
        
        best_choice = None
        best_utility = 0.0  # no purchase has utility = 0
        for j in in_stock_indices:
            util_j = u[j] + epsilons[j]
            if util_j > best_utility:  # outside option is 0
                best_utility = util_j
                best_choice = j
        
        # If best_choice is not None, product j is chosen
        if best_choice is not None:
            # Decrement inventory and add revenue
            inventory[best_choice] -= 1
            total_revenue += price[best_choice]
        # else no purchase
        # (We treat the outside good as the baseline with utility=0, so if best utility < 0 none is purchased.)
    
    profit = total_revenue - total_inventory_cost
    return profit

def average_profit(x, u, T, price, cost, mu=1.0, n_rep=1000, seed=None):
    """
    Estimate the average profit over n_rep simulation runs.
    Returns the mean and stdev.
    """
    profits = []
    # If you want reproducibility across the entire set of replications:
    if seed is not None:
        random.seed(seed)
    for rep in range(n_rep):
        # Use distinct seeds each replication for better coverage
        run_seed = None if seed is None else (seed + rep)
        p = simulate_mnl_choice(x, u, T, price, cost, mu=mu, seed=run_seed)
        profits.append(p)
    return statistics.mean(profits), statistics.pstdev(profits)

def simple_search(u, T, price, cost, mu=1.0, budget=1000):
    """
    A simple (and not necessarily optimal) search over possible inventory vectors.
    For demonstration, we only vary x_10 from 0..T, and set others to 0.
    You could generalize to more systematic or global search.
    """
    best_x = None
    best_estimated_profit = -1e9
    
    # We'll try x10 = 0..T (and x1..x9=0)
    # This automatically stays within budget if T <= 30 or so.
    
    for x10 in range(T+1):
        x_candidate = [0]*(len(u)-1) + [x10]  # all zero except the last product
        mean_p, _ = average_profit(x_candidate, u, T, price, cost, mu=mu, n_rep=50)
        if mean_p > best_estimated_profit:
            best_estimated_profit = mean_p
            best_x = x_candidate[:]
    
    return best_x, best_estimated_profit

if __name__ == "__main__":
    # Default values
    n = 10
    T = 30
    u = [5 + j for j in range(1,n+1)]   # base utilities = 6, 7, ..., 15
    price = [9]*n
    cost = [5]*n
    mu = 1.0
    
    # 1) Quick demonstration of the expected profit for x=[0,0,...,30]
    x_demo = [0]*(n-1) + [30]
    mean_profit, stdev_profit = average_profit(x_demo, u, T, price, cost, mu=mu, n_rep=1000, seed=123)
    print("Candidate x =", x_demo)
    print("Estimated Profit = {:.2f}, StdDev = {:.2f}".format(mean_profit, stdev_profit))
    
    # 2) Perform a simple search to see if there's any better solution
    best_x, best_estimated_profit = simple_search(u, T, price, cost, mu=mu, budget=1000)
    print("Best x found by simple search =", best_x)
    print("Best mean profit =", best_estimated_profit)

Candidate x = [0, 0, 0, 0, 0, 0, 0, 0, 0, 30]
Estimated Profit = 120.00, StdDev = 0.00
Best x found by simple search = [0, 0, 0, 0, 0, 0, 0, 0, 0, 30]
Best mean profit = 120.0


## o3-mini

In [16]:
completion = openai_client.chat.completions.create(
    model="o3-mini",
    messages=[
        {"role": "system", "content": "You are an expert in probability theory and stochastic modeling."},
        {"role": "user", "content": prompt},
    ]
)

# get the response
response = completion.choices[0].message.content

In [17]:
print(response)

Below is one way to “attack” the problem. In our write‐up we explain two ideas and then present a simulation–based “coordinate‐descent” search that stays within a fixed simulation–evaluation budget. (Note that with the MNL substitution structure an exact closed‐form answer is difficult because the sequence of stock–depletion events causes state–dependence. However, under additional “fluid” assumptions one may “approximate” by equating marginal cost with “critical” probabilities. Here we assume that simulation is our work–horse.)

────────────────────────────
1. Method Explanation
────────────────────────────
A. (Analytical / Approximate Reasoning—discussion only)

One might “roughly” imagine that with T customers arriving in sequence the expected number of purchases of product j depends on
  (i) its inherent “attraction” (base utility u^j = 5+j plus noise)
  (ii) whether higher–utility products are available.
Because every sale “uses up” some inventory (and because all products yield t

In [18]:
import numpy as np

def sample_gumbel(size=None, loc=0.0, scale=1.0):
    # NumPy has a function, but for clarity we call it here.
    return np.random.gumbel(loc=loc, scale=scale, size=size)

def simulate_profit(x, params, seed=None):
    """
    Given an inventory vector x (length n) and parameters in dict 'params',
    simulate one “day” (T customers arriving sequentially) and return profit.
    
    The profit is revenue from sale (price for product sold) minus cost incurred for initial inventory.
    """
    if seed is not None:
        np.random.seed(seed)
        
    n = params['num_prod']
    T = params['num_customer']
    base_utils = params['base_utils']  # a numpy array of length n: u^j = 5+j for j=1...n (we index 0...n-1)
    prices = params['price']           # array of length n
    costs = params['cost']             # array of length n
    
    # Incur fixed cost for ordering all items:
    total_cost = np.sum(np.array(x) * np.array(costs))
    
    # Copy inventory (so as not to change original x)
    inventory = np.array(x).copy()
    
    revenue = 0.0
    
    for t in range(T):
        # identify which products are available
        available_idx = np.where(inventory > 0)[0]
        
        # Calculate utilities for available products:
        utilities = {}
        # Always include the no–purchase option with utility 0
        utilities['np'] = 0.0
        
        # For each available product j (j from 0 to n-1)
        for j in available_idx:
            # sample a Gumbel epsilon and add base utility
            u_val = base_utils[j] + sample_gumbel()
            utilities[j] = u_val
        
        # Choose the option with maximum utility
        # If the maximum is a product, we record sale; else, no sale.
        best = max(utilities, key=utilities.get)
        if best != 'np':
            # a sale occurs: update revenue and decrement inventory for that product
            revenue += prices[best]
            inventory[best] -= 1
    profit = revenue - total_cost
    return profit


def evaluate_inventory(x, params, num_rep=100):
    """
    Evaluate the expected profit for inventory vector x by averaging over num_rep replications.
    """
    profits = []
    for rep in range(num_rep):
        # use different seed for each replication, e.g. rep number + a constant.
        profit = simulate_profit(x, params, seed=rep)
        profits.append(profit)
    # Return mean and standard error
    mean_profit = np.mean(profits)
    std_profit = np.std(profits) / np.sqrt(num_rep)
    return mean_profit, std_profit


def coordinate_descent_search(initial_x, params, num_rep=100, max_iter=20):
    """
    Repeatedly try increasing or decreasing one product's inventory by 1 to see if profit improves.
    """
    current_x = initial_x.copy()
    best_profit, se = evaluate_inventory(current_x, params, num_rep=num_rep)
    improvement = True
    iter_num = 0
    # We'll count evaluations to keep within our budget.
    evaluations = 1
    
    while improvement and iter_num < max_iter and evaluations < 1000:
        improvement = False
        iter_num += 1
        # loop over products (coordinate by coordinate)
        for j in range(len(current_x)):
            # Try increase by one:
            candidate = current_x.copy()
            candidate[j] += 1
            cand_profit, _ = evaluate_inventory(candidate, params, num_rep=num_rep)
            evaluations += 1
            if cand_profit > best_profit:
                best_profit = cand_profit
                current_x = candidate
                improvement = True
            else:
                # Try decreasing by one if possible
                if current_x[j] > 0:
                    candidate = current_x.copy()
                    candidate[j] -= 1
                    cand_profit, _ = evaluate_inventory(candidate, params, num_rep=num_rep)
                    evaluations += 1
                    if cand_profit > best_profit:
                        best_profit = cand_profit
                        current_x = candidate
                        improvement = True
        # Optionally print iteration summary:
        print(f"Iteration {iter_num}: x = {current_x}, avg profit = {best_profit:.2f} (evaluations: {evaluations})")
    return current_x, best_profit, evaluations

if __name__ == '__main__':
    # Parameter settings
    n = 10
    T = 30
    
    params = {
        'num_prod': n,
        'num_customer': T,
        'base_utils': np.array([5 + (j+1) for j in range(n)]),  # u^j = 5+j+? (we use j+1 for j=0,...,n-1)
        'price': [9]*n,
        'cost': [5]*n
    }
    
    # Starting guess: using the provided default initial inventory: (3,3,...,3)
    initial_x = [3] * n
    
    print("Evaluating baseline inventory x = [3,...,3]")
    base_profit, base_se = evaluate_inventory(initial_x, params, num_rep=100)
    print(f"Baseline avg profit = {base_profit:.2f} (standard error ~{base_se:.2f})")
    
    print("\nStarting coordinate descent search for an improved inventory vector...")
    best_x, best_profit, tot_evals = coordinate_descent_search(initial_x, params, num_rep=100, max_iter=20)
    
    print("\nFinal chosen inventory vector x:", best_x)
    print(f"Final expected profit (approx.) = {best_profit:.2f} (total evaluations: {tot_evals})")

Evaluating baseline inventory x = [3,...,3]
Baseline avg profit = 120.00 (standard error ~0.00)

Starting coordinate descent search for an improved inventory vector...
Iteration 1: x = [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], avg profit = 120.00 (evaluations: 21)

Final chosen inventory vector x: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Final expected profit (approx.) = 120.00 (total evaluations: 21)


## Claude 3.5 Sonnet

In [19]:
response = claude_client.messages.create(
    model="claude-3-5-sonnet-20241022",  # Or another Claude 3 model
    max_tokens=4096,
    system="You are an expert in probability theory and stochastic modeling.",
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(response.content[0].text)

I'll approach this problem using a simulation-based method since the dynamic substitution effects and stochastic customer choices make a closed-form solution challenging. Here's my solution:

1. METHOD EXPLANATION
I'll use a combination of simulation and optimization:
- Monte Carlo simulation to estimate expected profit for given inventory levels
- A gradient-free optimization approach (hill climbing with random restarts) to find good inventory levels
- Exploration of neighborhood solutions to avoid local optima

2. CODE IMPLEMENTATION

```python
import numpy as np
from typing import List, Tuple
import random

class RetailOptimizer:
    def __init__(self, 
                 num_prod: int = 10,
                 num_customer: int = 30,
                 mu: float = 1.0,
                 base_utilities: List[float] = None,
                 prices: List[float] = None,
                 costs: List[float] = None):
        
        self.num_prod = num_prod
        self.num_customer = num_custom

In [20]:
import numpy as np
from typing import List, Tuple
import random

class RetailOptimizer:
    def __init__(self, 
                 num_prod: int = 10,
                 num_customer: int = 30,
                 mu: float = 1.0,
                 base_utilities: List[float] = None,
                 prices: List[float] = None,
                 costs: List[float] = None):
        
        self.num_prod = num_prod
        self.num_customer = num_customer
        self.mu = mu
        
        # Initialize default values if not provided
        if base_utilities is None:
            self.base_utilities = [5 + j for j in range(num_prod)]
        else:
            self.base_utilities = base_utilities
            
        if prices is None:
            self.prices = [9.0] * num_prod
        else:
            self.prices = prices
            
        if costs is None:
            self.costs = [5.0] * num_prod
        else:
            self.costs = costs

    def generate_customer_choice(self, available_products: List[int]) -> int:
        """Simulate one customer choice using MNL model"""
        if not available_products:
            return 0  # No-purchase option
        
        # Generate Gumbel noise for available products and no-purchase option
        utilities = [0.0]  # No-purchase utility
        for j in available_products:
            epsilon = np.random.gumbel(loc=0, scale=self.mu)
            utilities.append(self.base_utilities[j] + epsilon)
            
        # Customer chooses maximum utility option
        max_utility_idx = np.argmax(utilities)
        if max_utility_idx == 0:
            return 0  # No purchase
        return available_products[max_utility_idx - 1]

    def simulate_profit(self, inventory: List[int], num_replications: int = 100) -> Tuple[float, float]:
        """Simulate expected profit for given inventory levels"""
        profits = []
        
        for _ in range(num_replications):
            current_inventory = inventory.copy()
            profit = -sum(self.costs[j] * inventory[j] for j in range(self.num_prod))
            
            for _ in range(self.num_customer):
                # Determine available products
                available = [j for j, inv in enumerate(current_inventory) if inv > 0]
                
                # Get customer choice
                choice = self.generate_customer_choice(available)
                
                if choice > 0:  # If purchase made
                    profit += self.prices[choice]
                    current_inventory[choice] -= 1
                    
            profits.append(profit)
            
        return np.mean(profits), np.std(profits)

    def optimize_inventory(self, budget: int = 1000):
        """Find optimal inventory levels using hill climbing with random restarts"""
        best_inventory = None
        best_profit = float('-inf')
        evals_used = 0
        
        while evals_used < budget:
            # Random starting point
            current_inv = [random.randint(0, 5) for _ in range(self.num_prod)]
            
            # Hill climbing
            improved = True
            while improved and evals_used < budget:
                improved = False
                
                # Try adjusting each product's inventory
                for j in range(self.num_prod):
                    for delta in [-1, 1]:
                        if current_inv[j] + delta >= 0:
                            new_inv = current_inv.copy()
                            new_inv[j] += delta
                            
                            mean_profit, _ = self.simulate_profit(new_inv, 50)
                            evals_used += 1
                            
                            if mean_profit > best_profit:
                                best_profit = mean_profit
                                best_inventory = new_inv.copy()
                                current_inv = new_inv.copy()
                                improved = True
                                
        return best_inventory, best_profit

# Example usage
optimizer = RetailOptimizer()
best_inv, best_profit = optimizer.optimize_inventory(budget=1000)

# Validate solution with more replications
final_profit, std_dev = optimizer.simulate_profit(best_inv, num_replications=200)

print(f"Best inventory levels: {best_inv}")
print(f"Expected profit: {final_profit:.2f} ± {1.96*std_dev/np.sqrt(200):.2f}")

Best inventory levels: [0, 5, 1, 5, 1, 5, 2, 1, 5, 5]
Expected profit: 120.00 ± 0.00


## DeepSeek-R1

In [21]:
import numpy as np

class ProfitSimulator:
    def __init__(self, num_prod, num_customer, u, p, c, mu, budget=1000):
        self.num_prod = num_prod
        self.num_customer = num_customer
        self.u = np.array(u)
        self.p = np.array(p)
        self.c = np.array(c)
        self.mu = mu
        self.budget = budget
        self.replications_used = 0

    def evaluate(self, x, num_replications):
        if self.replications_used + num_replications > self.budget:
            num_replications = self.budget - self.replications_used
        if num_replications <= 0:
            return None
        total_profit = 0.0
        x = np.array(x, dtype=int)
        for _ in range(num_replications):
            inventory = x.copy()
            sold = np.zeros_like(x)
            for _ in range(self.num_customer):
                available = np.where(inventory > 0)[0]
                utilities_available = []
                for j in available:
                    epsilon = np.random.gumbel(loc=0, scale=self.mu)
                    utilities_available.append(self.u[j] + epsilon)
                epsilon_0 = np.random.gumbel(loc=0, scale=self.mu)
                all_utilities = utilities_available + [epsilon_0]
                max_idx = np.argmax(all_utilities)
                if max_idx < len(available):
                    chosen_j = available[max_idx]
                    inventory[chosen_j] -= 1
                    sold[chosen_j] += 1
            revenue = np.sum(self.p * sold)
            cost_total = np.sum(self.c * x)
            profit = revenue - cost_total
            total_profit += profit
        avg_profit = total_profit / num_replications
        self.replications_used += num_replications
        return avg_profit

def optimize_inventory(num_prod=10, num_customer=30, u=None, p=None, c=None, mu=1.0, init_level=None, budget=1000):
    if u is None:
        u = [5 + (j + 1) for j in range(num_prod)]
    if p is None:
        p = [9] * num_prod
    if c is None:
        c = [5] * num_prod
    if init_level is None:
        x = [3] * num_prod
    else:
        x = init_level.copy()
    
    simulator = ProfitSimulator(num_prod, num_customer, u, p, c, mu, budget)
    current_profit = simulator.evaluate(x, num_replications=30)
    if current_profit is None:
        return x, 0.0
    
    improvement = True
    while improvement and simulator.replications_used < budget:
        improvement = False
        for j in range(num_prod):
            x_plus = x.copy()
            x_plus[j] += 1
            profit_plus = simulator.evaluate(x_plus, 30)
            if profit_plus is None:
                break
            
            profit_minus = None
            if x[j] > 0:
                x_minus = x.copy()
                x_minus[j] -= 1
                profit_minus = simulator.evaluate(x_minus, 30)
                if profit_minus is None:
                    break
            
            best_profit = current_profit
            best_x = x.copy()
            if profit_plus > best_profit:
                best_profit = profit_plus
                best_x = x_plus.copy()
            if profit_minus is not None and profit_minus > best_profit:
                best_profit = profit_minus
                best_x = x_minus.copy()
            
            if best_profit > current_profit:
                x = best_x.copy()
                current_profit = best_profit
                improvement = True
                break
    
    remaining = budget - simulator.replications_used
    if remaining > 0:
        final_eval = simulator.evaluate(x, remaining)
        if final_eval is not None:
            current_profit = (current_profit * (budget - remaining) + final_eval * remaining) / budget
    return x, current_profit

np.random.seed(0)
optimal_x, estimated_profit = optimize_inventory(budget=1000)
print("Optimal inventory:", optimal_x)
print("Estimated expected profit:", estimated_profit)

simulator_baseline = ProfitSimulator(10, 30, [5 + (j+1) for j in range(10)], [9]*10, [5]*10, 1.0, 1000)
baseline_profit = simulator_baseline.evaluate([0]*10, 1000)
print("Baseline (x=0) profit:", baseline_profit)

Optimal inventory: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Estimated expected profit: 119.982
Baseline (x=0) profit: 0.0
